# Import librairies

In [1]:
# To check PEP8 need installation : pip install flake8 pycodestyle pycodestyle_magic
# add %%flake8 in each block to check it
%load_ext pycodestyle_magic

In [3]:
# %%flake8

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate, KFold
from sklearn.metrics import fbeta_score, make_scorer

# Load data files

In [4]:
# %%flake8

data_train = pd.read_csv('data_train.csv')

# Overview of data

In [15]:
# To optimize display of the Dataframe

#increase max rows displayed to 999
pd.set_option('display.max_rows', 999)

#increase max columns displayed to 999
pd.set_option('display.max_columns', 999)
#increase columns width to 100
pd.set_option('display.max_colwidth', 200)

# Formating float
pd.options.display.float_format = '{:.2f}'.format

In [6]:
# %%flake8

# define random_state
rd_state = 16

**Train data**

In [7]:
data_train.head(10)

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATIO

In [14]:
data_train.describe(include='all')

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATIO

In [18]:
data_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 245 columns):
 #    Column                                             Non-Null Count   Dtype  
---   ------                                             --------------   -----  
 0    SK_ID_CURR                                         307511 non-null  int64  
 1    NAME_CONTRACT_TYPE                                 307511 non-null  int64  
 2    FLAG_OWN_CAR                                       307511 non-null  int64  
 3    FLAG_OWN_REALTY                                    307511 non-null  int64  
 4    CNT_CHILDREN                                       307511 non-null  int64  
 5    AMT_INCOME_TOTAL                                   307511 non-null  float64
 6    AMT_CREDIT                                         307511 non-null  float64
 7    AMT_ANNUITY                                        307499 non-null  float64
 8    AMT_GOODS_PRICE                                    307233 non-

In [19]:
print('data_train number of rows : {}' .format(data_train.shape[0]))
print('data_train number of columns : {}' .format(data_train.shape[1]))

data_train number of rows : 307511
data_train number of columns : 245


# Modelisation

In [20]:
#split data between features and target
X = data_train.drop(columns = ['TARGET'])
y = data_train['TARGET']

# Feature names
features = list(data_train.columns)

In [21]:
#Separation training / testing set use stratify due to classes are imbalanced
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state=rd_state , stratify=y)

In [22]:
#sampling X_train
X_sample_train, X_sample_test, y_sample_train, y_sample_test = train_test_split(X_train, y_train, train_size=0.2,random_state=rd_state , stratify=y_train)

In [34]:
# define pipeline

# oversampling of the majority class
over = SMOTE(sampling_strategy=0.1, random_state=rd_state)

# undersampling of the majority class
under = RandomUnderSampler(sampling_strategy=0.5, random_state=rd_state)

# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median')

# Scale each feature to 0-1
scaler = StandardScaler()

classifier = LogisticRegression(random_state=rd_state, max_iter=10000)


steps = [['imputer', imputer], ['scaler', scaler], ['classifier', classifier]]

pipeline = imbpipeline(steps=steps)


#dictionnary of score
# create a fbeta score
f2_scorer = make_scorer(fbeta_score, beta=2)
scores = {'AUC':'roc_auc','fbeta':f2_scorer,'precision_recall_curve':'average_precision'}

#define cv
cv=KFold(5,shuffle=True,random_state=rd_state)

#define parameters to test model LogisticRegression
param = {'classifier__C':[0.1, 1, 10, 100]}


#use function test_model to test ElasticNet
grid = GridSearchCV(estimator=pipeline, param_grid = param, cv=cv, return_train_score=True,scoring=scores,refit='precision_recall_curve',n_jobs=-1)
grid.fit(X_sample_train.to_numpy(),y_sample_train.to_numpy())

GridSearchCV(cv=KFold(n_splits=5, random_state=16, shuffle=True),
             estimator=Pipeline(steps=[['imputer',
                                        SimpleImputer(strategy='median')],
                                       ['scaler', StandardScaler()],
                                       ['classifier',
                                        LogisticRegression(max_iter=10000,
                                                           random_state=16)]]),
             n_jobs=-1, param_grid={'classifier__C': [0.1, 1, 10, 100]},
             refit='precision_recall_curve', return_train_score=True,
             scoring={'AUC': 'roc_auc',
                      'fbeta': make_scorer(fbeta_score, beta=2),
                      'precision_recall_curve': 'average_precision'})

In [35]:
#create function to display dataframe of the results
def display_result(grid,model):
    results = pd.DataFrame(grid.cv_results_)
    results.insert(0,'Model',model)
    display(results)
    return results

In [36]:
#use function display_result to display dataframe of the result for ElasticNet
results_logisticregression_imbalanced=display_result(grid,'LogisticRegression Imbalanced')

,Model,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,params,split0_test_AUC,split1_test_AUC,split2_test_AUC,split3_test_AUC,split4_test_AUC,mean_test_AUC,std_test_AUC,rank_test_AUC,split0_train_AUC,split1_train_AUC,split2_train_AUC,split3_train_AUC,split4_train_AUC,mean_train_AUC,std_train_AUC,split0_test_fbeta,split1_test_fbeta,split2_test_fbeta,split3_test_fbeta,split4_test_fbeta,mean_test_fbeta,std_test_fbeta,rank_test_fbeta,split0_train_fbeta,split1_train_fbeta,split2_train_fbeta,split3_train_fbeta,split4_train_fbeta,mean_train_fbeta,std_train_fbeta,split0_test_precision_recall_curve,split1_test_precision_recall_curve,split2_test_precision_recall_curve,split3_test_precision_recall_curve,split4_test_precision_recall_curve,mean_test_precision_recall_curve,std_test_precision_recall_curve,rank_test_precision_recall_curve,split0_train_precision_recall_curve,split1_train_precision_recall_curve,split2_train_precision_recall_curve,split3_train_precision_recall_curve,split4_train_precision_recall_curve,mean_train_precision_recall_curve,std_train_precision_recall_curve
0,LogisticRegression Imbalanced,6.67,0.55,0.37,0.05,0.10,{'classifier__C': 0.1},0.73,0.74,0.73,0.75,0.75,0.74,0.01,1,0.76,0.76,0.76,0.75,0.75,0.76,0.00,0.01,0.02,0.01,0.02,0.01,0.02,0.00,4,0.02,0.02,0.02,0.02,0.02,0.02,0.00,0.21,0.22,0.21,0.22,0.22,0.22,0.01,3,0.24,0.24,0.24,0.24,0.24,0.24,0.00
1,LogisticRegression Imbalanced,11.85,0.92,0.35,0.04,1,{'classifier__C': 1},0.73,0.74,0.73,0.75,0.75,0.74,0.01,2,0.76,0.76,0.76,0.76,0.75,0.76,0.00,0.02,0.02,0.02,0.02,0.01,0.02,0.00,3,0.02,0.02,0.02,0.02,0.02,0.02,0.00,0.21,0.22,0.21,0.22,0.22,0.22,0.01,4,0.24,0.24,0.24,0.24,0.24,0.24,0.00
2,LogisticRegression Imbalanced,16.85,1.66,0.34,0.02,10,{'classifier__C': 10},0.73,0.74,0.73,0.75,0.75,0.74,0.01,3,0.76,0.76,0.76,0.76,0.75,0.76,0.00,0.02,0.02,0.02,0.02,0.01,0.02,0.00,1,0.02,0.02,0.02,0.02,0.02,0.02,0.00,0.21,0.22,0.21,0.22,0.22,0.22,0.01,1,0.24,0.24,0.24,0.24,0.24,0.24,0.00
3,LogisticRegression Imbalanced,23.07,4.73,0.30,0.06,100,{'classifier__C': 100},0.73,0.74,0.73,0.75,0.75,0.74,0.01,4,0.76,0.76,0.76,0.76,0.75,0.76,0.00,0.02,0.02,0.02,0.02,0.01,0.02,0.00,2,0.02,0.02,0.02,0.02,0.02,0.02,0.00,0.21,0.22,0.21,0.22,0.22,0.22,0.00,2,0.24,0.24,0.24,0.24,0.24,0.24,0.00


In [37]:
#create function to display score and best params for a model
def display_score(result,grid):
    print('AUC score: %.3f' % result.loc[grid.best_index_]['mean_test_AUC'])
    print('fbeta score: %.3f' % result.loc[grid.best_index_]['mean_test_fbeta'])
    print('precision_recall_curve score: %.3f' % grid.best_score_)
    print('Best parameters: %s' % grid.best_params_)

In [38]:
#use function display_score to display score and best_params for ElasticNet
display_score(results_logisticregression_imbalanced,grid)

AUC score: 0.738
fbeta score: 0.018
precision_recall_curve score: 0.217
Best parameters: {'classifier__C': 10}


In [42]:
# define pipeline

# oversampling of the minority class
over = SMOTE(random_state=rd_state)

# undersampling of the majority class
under = RandomUnderSampler(random_state=rd_state)

# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median')

# Scale each feature to 0-1
scaler = StandardScaler()

classifier = LogisticRegression(random_state=rd_state, max_iter=10000)


steps = [['imputer', imputer], ['scaler', scaler],['over', over], ['under', under], ['classifier', classifier]]
# steps = [['over', over], ['under', under], ['imputer', imputer], ['scaler', scaler], ['classifier', classifier]]

pipeline = imbpipeline(steps=steps)


#dictionnary of score
# create a fbeta score
f2_scorer = make_scorer(fbeta_score, beta=2)
scores = {'AUC':'roc_auc','fbeta':f2_scorer,'precision_recall_curve':'average_precision'}

#define cv
cv=KFold(5,shuffle=True,random_state=rd_state)

#define parameters to test model LogisticRegression
param = {'classifier__C':[0.1, 1, 10, 100],
         'over__sampling_strategy': [0.1,0.2,0.3],
         'under__sampling_strategy': [0.4,0.5,0.6]}


#use function test_model to test ElasticNet
grid = GridSearchCV(estimator=pipeline, param_grid = param, cv=cv, return_train_score=True,scoring=scores,refit='precision_recall_curve',n_jobs=-1)
grid.fit(X_sample_train.to_numpy(),y_sample_train.to_numpy())

GridSearchCV(cv=KFold(n_splits=5, random_state=16, shuffle=True),
             estimator=Pipeline(steps=[['imputer',
                                        SimpleImputer(strategy='median')],
                                       ['scaler', StandardScaler()],
                                       ['over', SMOTE(random_state=16)],
                                       ['under',
                                        RandomUnderSampler(random_state=16)],
                                       ['classifier',
                                        LogisticRegression(max_iter=10000,
                                                           random_state=16)]]),
             n_jobs=-1,
             param_grid={'classifier__C': [0.1, 1, 10, 100],
                         'over__sampling_strategy': [0.1, 0.2, 0.3],
                         'under__sampling_strategy': [0.4, 0.5, 0.6]},
             refit='precision_recall_curve', return_train_score=True,
             scoring={'AUC': 'roc_a

In [43]:
#use function display_result to display dataframe of the result for ElasticNet
results_logisticregression=display_result(grid,'LogisticRegression')

,Model,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_over__sampling_strategy,param_under__sampling_strategy,params,split0_test_AUC,split1_test_AUC,split2_test_AUC,split3_test_AUC,split4_test_AUC,mean_test_AUC,std_test_AUC,rank_test_AUC,split0_train_AUC,split1_train_AUC,split2_train_AUC,split3_train_AUC,split4_train_AUC,mean_train_AUC,std_train_AUC,split0_test_fbeta,split1_test_fbeta,split2_test_fbeta,split3_test_fbeta,split4_test_fbeta,mean_test_fbeta,std_test_fbeta,rank_test_fbeta,split0_train_fbeta,split1_train_fbeta,split2_train_fbeta,split3_train_fbeta,split4_train_fbeta,mean_train_fbeta,std_train_fbeta,split0_test_precision_recall_curve,split1_test_precision_recall_curve,split2_test_precision_recall_curve,split3_test_precision_recall_curve,split4_test_precision_recall_curve,mean_test_precision_recall_curve,std_test_precision_recall_curve,rank_test_precision_recall_curve,split0_train_precision_recall_curve,split1_train_precision_recall_curve,split2_train_precision_recall_curve,split3_train_precision_recall_curve,split4_train_precision_recall_curve,mean_train_precision_recall_curve,std_train_precision_recall_curve
0,LogisticRegression,4.44,0.27,0.35,0.03,0.10,0.10,0.40,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.4}",0.72,0.74,0.73,0.74,0.75,0.74,0.01,1,0.76,0.75,0.76,0.75,0.75,0.75,0.00,0.30,0.30,0.27,0.32,0.31,0.30,0.02,36,0.33,0.32,0.33,0.32,0.31,0.32,0.01,0.21,0.20,0.20,0.22,0.22,0.21,0.01,5,0.23,0.23,0.23,0.23,0.23,0.23,0.00
1,LogisticRegression,4.70,0.18,0.35,0.06,0.10,0.10,0.50,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.5}",0.72,0.73,0.73,0.74,0.74,0.73,0.01,4,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.34,0.35,0.32,0.37,0.36,0.35,0.02,14,0.37,0.37,0.37,0.37,0.36,0.37,0.00,0.20,0.20,0.20,0.22,0.22,0.21,0.01,26,0.23,0.22,0.23,0.23,0.22,0.23,0.00
2,LogisticRegression,4.45,0.18,0.33,0.05,0.10,0.10,0.60,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.1, 'under__sampling_strategy': 0.6}",0.72,0.74,0.73,0.74,0.74,0.73,0.01,6,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.37,0.38,0.35,0.38,0.39,0.38,0.01,3,0.40,0.39,0.39,0.39,0.39,0.39,0.00,0.20,0.20,0.20,0.22,0.22,0.21,0.01,30,0.23,0.22,0.22,0.22,0.22,0.22,0.00
3,LogisticRegression,7.00,0.33,0.39,0.05,0.10,0.20,0.40,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.4}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,13,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.31,0.31,0.29,0.32,0.33,0.31,0.01,28,0.33,0.32,0.33,0.33,0.32,0.33,0.00,0.21,0.20,0.20,0.22,0.22,0.21,0.01,2,0.23,0.23,0.23,0.22,0.22,0.23,0.00
4,LogisticRegression,6.70,0.26,0.40,0.06,0.10,0.20,0.50,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.5}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,15,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.35,0.34,0.33,0.36,0.36,0.35,0.01,15,0.37,0.36,0.37,0.36,0.36,0.37,0.00,0.21,0.20,0.20,0.22,0.22,0.21,0.01,7,0.23,0.23,0.23,0.22,0.22,0.23,0.00
5,LogisticRegression,6.28,0.36,0.37,0.03,0.10,0.20,0.60,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.6}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,14,0.76,0.75,0.75,0.75,0.75,0.75,0.00,0.37,0.38,0.35,0.38,0.38,0.37,0.01,5,0.39,0.39,0.39,0.39,0.39,0.39,0.00,0.21,0.20,0.20,0.22,0.22,0.21,0.01,18,0.23,0.23,0.22,0.22,0.22,0.22,0.00
6,LogisticRegression,10.09,0.69,0.47,0.03,0.10,0.30,0.40,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.3, 'under__sampling_strategy': 0.4}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,25,0.75,0.75,0.75,0.75,0.75,0.75,0.00,0.31,0.31,0.28,0.32,0.32,0.31,0.01,32,0.34,0.32,0.33,0.33,0.33,0.33,0.00,0.21,0.21,0.20,0.22,0.22,0.21,0.01,6,0.23,0.23,0.23,0.22,0.22,0.22,0.00
7,LogisticRegression,9.97,0.22,0.47,0.03,0.10,0.30,0.50,"{'classifier__C': 0.1, 'over__sampling_strategy': 0.3, 'under__sampling_strategy': 0.5}",0.72,0.73,0.72,0.74,0.74,0.73,0.01,26,0.75,0.75,0.75,0.75,0.75,0.75,0.00,0.35,0.34,0.32,0.35,0.36,0.34,0.01,24,0.37,0.36,0.37,0.36,0.3

In [44]:
#use function display_score to display score and best_params for ElasticNet
display_score(results_logisticregression,grid)

AUC score: 0.731
fbeta score: 0.311
precision_recall_curve score: 0.211
Best parameters: {'classifier__C': 1, 'over__sampling_strategy': 0.2, 'under__sampling_strategy': 0.4}


In [ ]:
.